<a href="https://colab.research.google.com/github/nannthd/project_AIEngineer/blob/main/coconut_yolov8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Install Dependencies


In [ ]:
!pip install roboflow

In [ ]:
import roboflow
# clone YOLOv8 repository
!git clone https://github.com/autogyro/yolo-V8  # clone repo
%cd yolo-V8
%pip install -r requirements.txt #install dependencies

In [3]:
import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0) if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 2.3.0+cu121 (CPU)


#Download Custom Dataset

In [4]:
%cd /content/yolo-V8

/content/yolo-V8


In [5]:
from roboflow import Roboflow
rf = Roboflow(api_key="AWtZRbYK04sFZNNZ1t4X")
project = rf.workspace("projectaiengineer").project("coconuts_dataset")
version = project.version(4)
dataset = version.download("yolov8")

loading Roboflow workspace...
loading Roboflow project...
Dependency ultralytics==8.0.196 is required but found version=8.0.8, to fix: `pip install ultralytics==8.0.196`



Extracting Dataset Version Zip to coconuts_dataset-4 in yolov8:: 100%|██████████| 658/658 [00:00<00:00, 4161.62it/s]


In [6]:
# this is the YAML file Roboflow wrote for us that we're loading into this notebook with our data
%cat {dataset.location}/data.yaml

names:
- coconuts
nc: 1
roboflow:
  license: CC BY 4.0
  project: coconuts_dataset
  url: https://universe.roboflow.com/projectaiengineer/coconuts_dataset/dataset/4
  version: 4
  workspace: projectaiengineer
test: ../test/images
train: coconuts_dataset-4/train/images
val: coconuts_dataset-4/valid/images


# Define Model Configuration and Architecture

In [7]:
# define number of classes based on YAML
import yaml
with open(dataset.location + "/data.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [8]:
#this is the model configuration we will use for our tutorial
%cat /content/yolo-V8/ultralytics/models/v8/yolov8m.yaml

# Ultralytics YOLO 🚀, GPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # scales module repeats
width_multiple: 0.75  # scales convolution channels

# YOLOv8.0m backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [768, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [768, True]]
  - [-1, 1, SPPF, [768, 5]]  # 9

# YOLOv8.0m head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 13

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 17 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 20

In [9]:
#customize iPython writefile so we can write variables
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [11]:
%mkdir -p /content/yolo-V8/models

In [12]:
%%writetemplate /content/yolo-V8/models/custom_yolov8m.yaml

# Parameters
nc: 80  # number of classes
depth_multiple: 0.67  # scales module repeats
width_multiple: 0.75  # scales convolution channels

# YOLOv8.0m backbone
backbone:
  # [from, repeats, module, args]
  - [-1, 1, Conv, [64, 3, 2]]  # 0-P1/2
  - [-1, 1, Conv, [128, 3, 2]]  # 1-P2/4
  - [-1, 3, C2f, [128, True]]
  - [-1, 1, Conv, [256, 3, 2]]  # 3-P3/8
  - [-1, 6, C2f, [256, True]]
  - [-1, 1, Conv, [512, 3, 2]]  # 5-P4/16
  - [-1, 6, C2f, [512, True]]
  - [-1, 1, Conv, [768, 3, 2]]  # 7-P5/32
  - [-1, 3, C2f, [768, True]]
  - [-1, 1, SPPF, [768, 5]]  # 9

# YOLOv8.0m head
head:
  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 6], 1, Concat, [1]]  # cat backbone P4
  - [-1, 3, C2f, [512]]  # 13

  - [-1, 1, nn.Upsample, [None, 2, 'nearest']]
  - [[-1, 4], 1, Concat, [1]]  # cat backbone P3
  - [-1, 3, C2f, [256]]  # 17 (P3/8-small)

  - [-1, 1, Conv, [256, 3, 2]]
  - [[-1, 12], 1, Concat, [1]]  # cat head P4
  - [-1, 3, C2f, [512]]  # 20 (P4/16-medium)

  - [-1, 1, Conv, [512, 3, 2]]
  - [[-1, 9], 1, Concat, [1]]  # cat head P5
  - [-1, 3, C2f, [768]]  # 23 (P5/32-large)

  - [[15, 18, 21], 1, Detect, [nc]]  # Detect(P3, P4, P5)

# Train Custom YOLOv5 Detector

In [ ]:
!pip install ultralytics

In [22]:
# train yolov5s on custom data for ... epochs
# time its performance
%%time
%cd /content/yolo-V8
!python /content/yolo-V8/ultralytics/yolo/v8/detect/train.py --img 640 --batch 16 --epochs 10 --data {dataset.location}/data.yaml --cfg ./content/yolo-V8/models/custom_yolov8m.yaml --weights 'yolov8m.pt' --optimizer SGD --freeze 10 --image-weights --hyp ./data/hyps/hyp.scratch-high.yaml --name yolov5m_results  --cache

/content/yolo-V8
WARNING ⚠️ Ultralytics settings reset to default values. This may be due to a possible problem with your settings or a recent ultralytics package update. 
View settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.yaml'
Update settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Traceback (most recent call last):
  File "/content/yolo-V8/ultralytics/yolo/v8/detect/train.py", line 10, in <module>
    from ultralytics.yolo import v8
ModuleNotFoundError: No module named 'ultralytics.yolo'
CPU times: user 42.7 ms, sys: 5.5 ms, total: 48.2 ms
Wall time: 4.12 s


In [ ]:
from ultralytics import YOLO

# Load a model
model = YOLO("yolov8n.yaml")  # build a new model from scratch
model = YOLO("yolov8n.pt")  # load a pretrained model (recommended for training)

In [24]:
# Load your custom data.yaml file
data_yaml_path = "/content/yolo-V8/coconuts_dataset-4/data.yaml"
data = load_yaml(data_yaml_path)

# Check and update 'data' dictionary with necessary keys
data['endswith'] = '.jpg'  # Example key that may be missing

# Train the model
epochs = 10
model.train(data=data, epochs=epochs)

Ultralytics YOLOv8.0.8 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data={'names': ['coconuts'], 'nc': 1, 'roboflow': {'license': 'CC BY 4.0', 'project': 'coconuts_dataset', 'url': 'https://universe.roboflow.com/projectaiengineer/coconuts_dataset/dataset/4', 'version': 4, 'workspace': 'projectaiengineer'}, 'test': '../test/images', 'train': 'coconuts_dataset-4/train/images', 'val': 'coconuts_dataset-4/valid/images', 'endswith': '.jpg'}, epochs=10, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=ultralytics/assets/, show=Fa

TypeError: 'str' object is not callable

In [19]:
# Use the model
%cd /content/yolo-V8
results = model.train(data=f"{dataset.location}/data.yaml", epochs=10)  # train the model

Ultralytics YOLOv8.0.8 🚀 Python-3.10.12 torch-2.3.0+cu121 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/yolo-V8/coconuts_dataset-4/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, cache=False, device=, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=False, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, overlap_mask=True, mask_ratio=4, dropout=False, val=True, save_json=False, save_hybrid=False, conf=0.001, iou=0.7, max_det=300, half=True, dnn=False, plots=True, source=ultralytics/assets/, show=False, save_txt=False, save_conf=False, save_crop=False, hide_labels=False, hide_conf=False, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms=False, retina_masks=False, format=torchscript, keras=False, optimize=False, int8=False, dynamic=False, simplify=False, opset=17, workspace=4, nms=False, 

/content/yolo-V8


FileNotFoundError: Dataset not found ❌